In [1]:
import cobra
from cobra.io import load_model

# "iJO1366" and "salmonella" are also valid arguments
model = load_model("textbook")
model

Name,e_coli_core
Memory address,284bb1bbfd0
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


In [2]:
print("Reactions", len(model.reactions))
print("Metabolites", len(model.metabolites))
print("Genes", len(model.genes))

Reactions 95
Metabolites 72
Genes 137


In [3]:
model.reactions[12]

Reaction identifier,Biomass_Ecoli_core
Name,Biomass Objective Function with GAM
Memory address,0x284bc2117c0
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose 6-phosphate + 0.2557...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [4]:
for i in range(0, 10):
    print(model.metabolites[i])

13dpg_c
2pg_c
3pg_c
6pgc_c
6pgl_c
ac_c
ac_e
acald_c
acald_e
accoa_c


In [5]:
model.metabolites.get_by_id("ac_e")

## Exchange reaction
#  as mentioned with EX_
#  sink -> not sure how it gets out, magical secretion
#  DM -> magically pop in to the cytosol, converted metabolically to the rest, magical uptake

Metabolite identifier,ac_e
Name,Acetate
Memory address,0x284bb1cd700
Formula,C2H3O2
Compartment,e
In 2 reaction(s),"ACt2r, EX_ac_e"


In [6]:
pgi = model.reactions.get_by_id("PGI")
pgi

# mmol / gDw / h 
# Be mindful of convertions

Reaction identifier,PGI
Name,glucose-6-phosphate isomerase
Memory address,0x284bc2a2c70
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b4025
Lower bound,-1000.0
Upper bound,1000.0


In [7]:
print(pgi.name)
print(pgi.reaction)

glucose-6-phosphate isomerase
g6p_c <=> f6p_c


In [8]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-1000.0 < pgi < 1000.0
True


In [9]:
old_bounds = pgi.bounds
pgi.bounds = (0, 1000.0)
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print("Reversibility after modification:", pgi.reversibility)
pgi.bounds = old_bounds
print("Reversibility after resetting:", pgi.reversibility)

0 < pgi < 1000.0
Reversibility after modification: False
Reversibility after resetting: True


In [10]:
old_bounds = pgi.bounds
print('Upper bound prior to setting new lower bound:', pgi.upper_bound)
pgi.lower_bound = 1100
print('Upper bound after setting new lower bound:', pgi.upper_bound)
pgi.bounds = old_bounds

Upper bound prior to setting new lower bound: 1000.0


ValueError: The lower bound must be less than or equal to the upper bound (1100 <= 1000.0).

In [11]:
pgi.check_mass_balance()

{}

In [12]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

In [13]:
pgi.check_mass_balance()

{'charge': -1.0, 'H': -1.0}

In [14]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


In [15]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

'g6p_c <=> f6p_c'

In [16]:
atp = model.metabolites.get_by_id("atp_c")
atp

Metabolite identifier,atp_c
Name,ATP
Memory address,0x284bb1cda30
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"PPCK, PGK, ADK1, PYK, GLNS, PFK, ATPS4r, PPS, GLNabc, SUCOAS, ACKr, Biomass_Ecoli_core, ATPM"


In [17]:
print(atp.name)
print(atp.compartment)

ATP
c


In [18]:
atp.charge

-4

In [ ]:
print(atp.formula)

C10H12N5O13P3


In [20]:
len(atp.reactions)

13

In [21]:
model.metabolites.get_by_id("g6p_c").reactions

frozenset({<Reaction Biomass_Ecoli_core at 0x284bc2117c0>,
           <Reaction G6PDH2r at 0x284bc25ba00>,
           <Reaction GLCpts at 0x284bc261b20>,
           <Reaction PGI at 0x284bc2a2c70>})

In [22]:
gpr = pgi.gpr
print(gpr)
gpr_string = pgi.gene_reaction_rule
print(gpr_string)

b4025
b4025


In [23]:
pgi.genes

frozenset({<Gene b4025 at 0x284bc201730>})

In [24]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

Gene identifier,b4025
Name,pgi
Memory address,0x284bc201730
Functional,True
In 1 reaction(s),PGI


In [25]:
pgi_gene.reactions

frozenset({<Reaction PGI at 0x284bc2a2c70>})

In [26]:
model = load_model('textbook')
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


In [27]:
[reaction.bounds for reaction in model.reactions[:5]]

[(-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0)]

In [28]:
print('original objective: ', model.objective.expression)
with model:
    model.objective = 'ATPM'
    print('print objective in first context:', model.objective.expression)
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)
print('back to original objective:', model.objective.expression)

original objective:  1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
print objective in first context: 1.0*ATPM - 1.0*ATPM_reverse_5b752
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: 1.0*ATPM - 1.0*ATPM_reverse_5b752
back to original objective: 1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


In [29]:
with model as inner:
    inner.reactions.PFK.knock_out

In [30]:
from cobra import Model, Reaction, Metabolite

model = Model('example_model')

reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x284bd3f4160
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [31]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

ACP_c

Metabolite identifier,ACP_c
Name,acyl-carrier-protein
Memory address,0x284bd5062b0
Formula,C11H21N2O7PRS
Compartment,c
In 0 reaction(s),


In [32]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})
reaction.reaction

'ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c'

In [33]:
###### 

In [34]:
from cobra.io import load_model
model = load_model("textbook")
model

Name,e_coli_core
Memory address,284bd506b80
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


In [35]:
solution = model.optimize()
print(solution)
print(solution.objective_value)

<Solution 0.874 at 0x284bd5068e0>
0.8739215069684279


In [36]:
%%time
model.optimize().objective_value

CPU times: total: 0 ns
Wall time: 0 ns


0.8739215069684305

In [37]:
%%time
model.slim_optimize()

CPU times: total: 0 ns
Wall time: 0 ns


0.8739215069684305

In [38]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [39]:
model.metabolites.nadh_c.summary()

Percent,Flux,Reaction,Definition
13.14%,5.064,AKGDH,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
8.04%,3.1,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
41.58%,16.02,GAPD,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13.14%,5.064,MDH,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
24.09%,9.283,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-38.53,NADH16,4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q8h2_c


In [40]:
model.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
66.58%,45.51,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
23.44%,16.02,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
2.57%,1.758,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
7.41%,5.064,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
Percent,Flux,Reaction,Definition
12.27%,-8.39,ATPM,atp_c + h2o_c --> adp_c + h_c + pi_c
76.46%,-52.27,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
0.33%,-0.2235,GLNS,atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c
10.94%,-7.477,PFK,atp_c + f6p_c --> adp_c + fdp_c + h_c


In [41]:
biomass_rxn = model.reactions.get_by_id("Biomass_Ecoli_core")
biomass_rxn

Reaction identifier,Biomass_Ecoli_core
Name,Biomass Objective Function with GAM
Memory address,0x284bd538dc0
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose 6-phosphate + 0.2557...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [42]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction Biomass_Ecoli_core at 0x284bd538dc0>: 1.0}

In [43]:
# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x284bd538970>: 1.0}

In [44]:
model.optimize().objective_value

175.00000000000006

In [45]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model, model.reactions[:10])

,minimum,maximum
ACALD,-9.375513e-15,0.000000e+00
ACALDt,-6.214834e-15,0.000000e+00
ACKr,-1.832929e-14,1.039260e-14
ACONTa,2.000000e+01,2.000000e+01
ACONTb,2.000000e+01,2.000000e+01
ACt2r,-3.615752e-15,0.000000e+00
ADK1,0.000000e+00,1.924093e-13
AKGDH,2.000000e+01,2.000000e+01
AKGt2r,2.522043e-15,0.000000e+00
ALCD2x,-2.048926e-15,0.000000e+00


In [46]:
cobra.flux_analysis.flux_variability_analysis(
    model, model.reactions[:10], fraction_of_optimum=0.9)

,minimum,maximum
ACALD,-2.692308,0.000000e+00
ACALDt,-2.692308,0.000000e+00
ACKr,-4.117647,1.039260e-14
ACONTa,8.461538,2.000000e+01
ACONTb,8.461538,2.000000e+01
ACt2r,-4.117647,0.000000e+00
ADK1,0.000000,1.750000e+01
AKGDH,2.500000,2.000000e+01
AKGt2r,-1.489362,0.000000e+00
ALCD2x,-2.333333,0.000000e+00


In [47]:
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=False)

,minimum,maximum
FRD7,0.0,980.0
SUCDi,20.0,1000.0


In [48]:
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=True)

,minimum,maximum
FRD7,0.0,0.0
SUCDi,20.0,20.0


In [49]:
model.optimize()
model.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,[9.5; 10],6,100.00%
o2_e,EX_o2_e,60,[55.88; 60],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ac_e,EX_ac_e,0,[-2.059; 0],2,0.00%
acald_e,EX_acald_e,0,[-1.346; 0],2,0.00%
akg_e,EX_akg_e,0,[-0.7447; 0],5,0.00%
co2_e,EX_co2_e,-60,[-60; -54.17],1,100.00%
etoh_e,EX_etoh_e,0,[-1.167; 0],2,0.00%
for_e,EX_for_e,0,[-5.833; 0],1,0.00%
glu__L_e,EX_glu__L_e,0,[-0.6731; 0],5,0.00%


In [50]:
model.metabolites.pyr_c.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
50.00%,10,[9.5; 10],GLCpts,glc__D_e + pep_c --> g6p_c + pyr_c
0.00%,0,[-1.129; 0],LDH_D,lac__D_c + nad_c <=> h_c + nadh_c + pyr_c
0.00%,0,[0; 8.75],ME1,mal__L_c + nad_c --> co2_c + nadh_c + pyr_c
0.00%,0,[0; 8.75],ME2,mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
50.00%,10,[1.25; 18.75],PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
0.00%,0,[-1.346; 0],PYRt2,h_e + pyr_e <=> h_c + pyr_c
Percent,Flux,Range,Reaction,Definition
0.00%,0,[-0.1316; 0],Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
100.00%,-20,[-28.75; -13],PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
0.00%,0,[-5.833; 0],PFL,coa_c + pyr_c --> accoa_c + for_c


In [51]:
geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
geometric_fba_sol

,fluxes,reduced_costs
ACALD,0.000000e+00,0.0
ACALDt,0.000000e+00,0.0
ACKr,2.232750e-14,0.0
ACONTa,2.000000e+01,0.0
ACONTb,2.000000e+01,0.0
...,...,...
TALA,0.000000e+00,0.0
THD2,0.000000e+00,0.0
TKT1,0.000000e+00,0.0
TKT2,0.000000e+00,0.0


In [52]:
###

In [53]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = load_model("textbook")
ecoli_model = load_model("iJO1366")

In [54]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x284c5132a60>
pfk knocked out:  <Solution 0.704 at 0x284c5132700>


In [55]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x284c51325e0>
pfkA knocked out:  <Solution 0.874 at 0x284c51326d0>
pfkB knocked out:  <Solution 0.704 at 0x284c5132550>


In [56]:
deletion_results = single_gene_deletion(cobra_model)

In [57]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

,ids,growth,status
0,{b0351},0.873922,optimal
1,{b0118},0.873922,optimal
2,{b0116},0.782351,optimal
3,{b3736},0.374230,optimal
4,{b3115},0.873922,optimal
5,{s0001},0.211141,optimal
6,{b0356},0.873922,optimal
7,{b1849},0.873922,optimal
8,{b1276},0.873922,optimal
9,{b1241},0.873922,optimal


In [58]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

,ids,growth,status
0,{D_LACt2},8.739215e-01,optimal
1,{ACALD},8.739215e-01,optimal
2,{ACONTb},-1.376403e-15,optimal
3,{ETOHt2r},8.739215e-01,optimal
4,{ALCD2x},8.739215e-01,optimal
5,{CO2t},4.616696e-01,optimal
6,{ADK1},8.739215e-01,optimal
7,{ACKr},8.739215e-01,optimal
8,{Biomass_Ecoli_core},0.000000e+00,optimal
9,{AKGDH},8.583074e-01,optimal


In [59]:
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:]).round(4)

,ids,growth,status
0,{b0008},0.8739,optimal
1,"{b0008, b2935}",0.8739,optimal
2,"{b2465, b2464}",0.8739,optimal
3,"{b2465, b3919}",0.7040,optimal
4,{b2465},0.8739,optimal
5,"{b2465, b2935}",0.0000,optimal
6,"{b3919, b0008}",0.7040,optimal
7,{b2464},0.8739,optimal
8,{b2935},0.8739,optimal
9,"{b3919, b2935}",0.7040,optimal


In [ ]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())
print("###")
single = single_reaction_deletion(cobra_model)
double = double_reaction_deletion(cobra_model)

print(single.knockout["ATPM"])
print(double.knockout[{"ATPM", "TKT1"}])
print("###")
atpm = cobra_model.reactions.ATPM
tkt1 = cobra_model.reactions.TKT1
pfk = cobra_model.reactions.PFK

print(single.knockout[atpm, tkt1, pfk])
print(double.knockout[{atpm, tkt1}, {atpm, pfk}, {atpm}])


In [ ]:
####

In [ ]:
from cobra.io import load_model
from cobra.flux_analysis import production_envelope

model = load_model("textbook")

In [ ]:
prod_env = production_envelope(model, ["EX_glc__D_e", "EX_o2_e"])
prod_env.head()

In [ ]:
prod_env = production_envelope(
    model, ["EX_o2_e"], objective="EX_ac_e", carbon_sources="EX_glc__D_e")
prod_env.head()

In [ ]:
###

In [ ]:
from cobra.io import load_model
from cobra.sampling import sample

model = load_model("textbook")
s = sample(model, 100)
s.head()

In [ ]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

In [ ]:
s = sample(model, 100, method="achr")
s

In [ ]:
from cobra.sampling import OptGPSampler, ACHRSampler

In [ ]:
achr = ACHRSampler(model, thinning=10)
achr

In [ ]:
optgp = OptGPSampler(model, processes=4)
optgp

In [ ]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
s1 = achr.validate(np.atleast_2d(bad))

In [ ]:
counts = [np.mean(s.Biomass_Ecoli_core > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(
    np.mean(counts) * 100.0, np.std(counts) * 100.0))

In [ ]:
co = model.problem.Constraint(model.reactions.Biomass_Ecoli_core.flux_expression, lb=0.1)
model.add_cons_vars([co])

In [ ]:
s = sample(model, 10)
print(s.Biomass_Ecoli_core)

In [ ]:
from cobra.io import load_model
from cobra import Reaction, Metabolite, Model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

salmonella = load_model('salmonella')
nominal = salmonella.optimize()
loopless = loopless_solution(salmonella)
print(loopless)

In [ ]:
model = Model()
model.add_metabolites([Metabolite(i) for i in "ABC"])
model.add_reactions([Reaction(i) for i in ["EX_A", "DM_C", "v1", "v2", "v3"]])

model.reactions.EX_A.add_metabolites({"A": 1})
model.reactions.DM_C.add_metabolites({"C": -1})

model.reactions.v1.add_metabolites({"A": -1, "B": 1})
model.reactions.v2.add_metabolites({"B": -1, "C": 1})
model.reactions.v3.add_metabolites({"C": -1, "A": 1})

model.objective = 'DM_C'

In [ ]:
with model:
    add_loopless(model)
    solution = model.optimize()
print("loopless solution: status = " + solution.status)
print("loopless solution flux: v3 = %.1f" % solution.fluxes["v3"])

In [ ]:
solution = pfba(model)
print("parsimonious solution: status = " + solution.status)
print("loopless solution flux: v3 = %.1f" % solution.fluxes["v3"])

In [ ]:
model.reactions.v3.lower_bound = 1
with model:
    add_loopless(model)
    try:
        solution = model.optimize()
    except:
        print('model is infeasible')

In [ ]:
solution = pfba(model)
print("parsimonious solution: status = " + solution.status)
print("loopless solution flux: v3 = %.1f" % solution.fluxes["v3"])

In [ ]:
###

In [ ]:
import cobra

In [ ]:
test_model = cobra.Model("test_model")
v1 = cobra.Reaction("v1")
v2 = cobra.Reaction("v2")
v3 = cobra.Reaction("v3")
v4 = cobra.Reaction("v4")
v5 = cobra.Reaction("v5")
v6 = cobra.Reaction("v6")

test_model.add_reactions([v1, v2, v3, v4, v5, v6])

v1.reaction = "-> 2 A"
v2.reaction = "A <-> B"
v3.reaction = "A -> D"
v4.reaction = "A -> C"
v5.reaction = "C -> D"
v6.reaction = "D ->"

v1.bounds = (0.0, 3.0)
v2.bounds = (-3.0, 3.0)
v3.bounds = (0.0, 3.0)
v4.bounds = (0.0, 3.0)
v5.bounds = (0.0, 3.0)
v6.bounds = (0.0, 3.0)

test_model.objective = v6

In [ ]:
cobra.flux_analysis.find_blocked_reactions(test_model)

In [ ]:
consistent_model = cobra.flux_analysis.fastcc(test_model)
consistent_model.reactions

In [ ]:
consistent_model.objective

In [ ]:
###

In [ ]:
from cobra.io import load_model
from cobra.flux_analysis import gapfill
model = load_model("iYS1720")

In [ ]:
universal = cobra.Model("universal_reactions")
for i in [i.id for i in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(i)
    print(reaction)
    universal.add_reactions([reaction.copy()])
    model.remove_reactions([reaction])

In [ ]:
model.optimize().objective_value

In [ ]:
solution = gapfill(model, universal, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

In [ ]:
result = gapfill(model, universal, demand_reactions=False, iterations=4)
for i, entries in enumerate(result):
    print("---- Run %d ----" % (i + 1))
    for e in entries:
        print(e.id)

In [ ]:
with model:
    model.objective = model.add_boundary(model.metabolites.f6p_c, type='demand')
    solution = gapfill(model, universal)
    for reaction in solution[0]:
        print(reaction.id)
model.summary()

In [ ]:
###

In [ ]:
from cobra.io import load_model

model = load_model("textbook")
model.medium

In [ ]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

In [ ]:
model.slim_optimize()

In [ ]:
model.medium["EX_co2_e"] = 0.0
model.medium

In [ ]:
medium = model.medium
medium["EX_co2_e"] = 0.0
model.medium = medium

model.medium  # now it worked

In [ ]:
model = load_model("textbook")

with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    print(model.slim_optimize())
print(model.slim_optimize())
model.medium

In [ ]:
model.summary(fva=0.95)

In [ ]:
model.reactions.EX_succ_e
print("pre", model.optimize())

with model:
    model.reactions.EX_h2o_e.knock_out()
    print("random kockout", model.optimize())
    print("random kockout + slim", model.slim_optimize())
#     print(model.reactions.EX_co2_e.summary())
    
print(model.summary())

In [ ]:
with model:
    model.reactions.EX_ac_e.knock_out()
    model.optimize()
    print(model.summary())

In [ ]:
model.medium

In [ ]:
model = load_model("textbook")

med_copy = model.medium
med_copy['EX_o2_e'] = 0
# med_copy['EX_o2_e'] = 1000
model.medium = med_copy
# model.medium = model_medium

model.medium

In [ ]:
model.optimize()

In [ ]:
difference = model.problem.Variable('difference')

constraint = model.problem.Constraint(
    model.reactions.EX_glc__D_e.flux_expression -
    model.reactions.EX_nh4_e.flux_expression - difference,
    lb=-1000,
    ub=-1000)
model.add_cons_vars([difference, constraint])
print(constraint)

model.optimize()
model.summary()

In [ ]:
model = load_model('textbook')
# cobra.io.read_sbml_model()
# read_sbm model
with model:
    model.objective = {model.reactions.Biomass_Ecoli_core: 1}
    model.optimize()
    print(model.reactions.Biomass_Ecoli_core.flux)